In [1]:
import time
import glob

import glow
spark = glow.register(spark)
import pyspark.sql.functions as fx
from pyspark.sql.types import *
from pyspark.sql.functions import *
from random import sample

In [2]:
%%time
def load_datalake(path):
    delta_vcf = spark.read.format("delta").load(path)    
    return delta_vcf

multi_lake_path = "delta/test-delta-multi133"
delta_vcf133 = load_datalake(multi_lake_path)

CPU times: user 15.7 ms, sys: 3.35 ms, total: 19 ms
Wall time: 12.9 s


In [3]:
def get_variants_per_sample(df):
    return df.groupBy('sampleId').count().collect()

def count_min_pos_per_chromosome(df):
    return df.groupBy('sampleId', 'contigName').agg(fx.min(df.start)).collect()

def get_to_G_subs(df):
    return df.filter(array_contains(col("alternateAlleles"), 'G'))

def get_to_G_subs_count_per_sample(df):
    return df.filter(array_contains(col("alternateAlleles"), 'G')).groupBy('sampleId').count().collect()

In [4]:
%%time
delta_vcf133.count()

CPU times: user 5.29 ms, sys: 3.44 ms, total: 8.73 ms
Wall time: 3.86 s


2210827

In [5]:
%%time  
get_variants_per_sample(delta_vcf133)

CPU times: user 20.3 ms, sys: 4.8 ms, total: 25.1 ms
Wall time: 4.52 s


[Row(sampleId='CPCT02290028T', count=9735),
 Row(sampleId='CPCT02120028T', count=17202),
 Row(sampleId='CPCT02340042T', count=15600),
 Row(sampleId='CPCT02060142T', count=13583),
 Row(sampleId='CPCT02020434T', count=17878),
 Row(sampleId='CPCT02030278T', count=19379),
 Row(sampleId='CPCT02110078T', count=19184),
 Row(sampleId='CPCT02030408T', count=8060),
 Row(sampleId='CPCT02300026T', count=23464),
 Row(sampleId='CPCT02020659T', count=10117),
 Row(sampleId='CPCT02120153T', count=17881),
 Row(sampleId='CPCT02040192T', count=13637),
 Row(sampleId='CPCT02220079T', count=19393),
 Row(sampleId='CPCT02060120T', count=32151),
 Row(sampleId='CPCT02330084T', count=9693),
 Row(sampleId='CPCT02030498T', count=11081),
 Row(sampleId='CPCT02080198T', count=6591),
 Row(sampleId='CPCT02020593T', count=21695),
 Row(sampleId='CPCT02070348T', count=19476),
 Row(sampleId='CPCT02100119T', count=16874),
 Row(sampleId='CPCT02080024T', count=17475),
 Row(sampleId='CPCT02020572T', count=23598),
 Row(sampleId=

In [6]:
%%time
count_min_pos_per_chromosome(delta_vcf133)

CPU times: user 30.1 ms, sys: 18 ms, total: 48.1 ms
Wall time: 4.76 s


[Row(sampleId='CPCT02100030T', contigName='GL000204.1', min(start)=40453),
 Row(sampleId='CPCT02050203T', contigName='12', min(start)=184327),
 Row(sampleId='CPCT02020466T', contigName='MT', min(start)=7886),
 Row(sampleId='CPCT02270023T', contigName='12', min(start)=89793),
 Row(sampleId='CPCT02080024T', contigName='GL000193.1', min(start)=3900),
 Row(sampleId='CPCT02020741T', contigName='GL000231.1', min(start)=12016),
 Row(sampleId='CPCT02020741T', contigName='GL000220.1', min(start)=5027),
 Row(sampleId='CPCT02020516T', contigName='17', min(start)=91847),
 Row(sampleId='CPCT02030296T', contigName='13', min(start)=19058701),
 Row(sampleId='CPCT02030296T', contigName='6', min(start)=365600),
 Row(sampleId='CPCT02130051T', contigName='21', min(start)=9568964),
 Row(sampleId='CPCT02270023T', contigName='GL000213.1', min(start)=86738),
 Row(sampleId='CPCT02270023T', contigName='GL000216.1', min(start)=15365),
 Row(sampleId='CPCT02060196T', contigName='7', min(start)=225974),
 Row(sample

In [7]:
%%time
#get_to_G_subs(delta_vcf30).count()
get_to_G_subs_count_per_sample(delta_vcf133)

CPU times: user 15.8 ms, sys: 4.88 ms, total: 20.7 ms
Wall time: 5.44 s


[Row(sampleId='CPCT02290028T', count=1885),
 Row(sampleId='CPCT02120028T', count=4009),
 Row(sampleId='CPCT02340042T', count=3060),
 Row(sampleId='CPCT02060142T', count=2979),
 Row(sampleId='CPCT02020434T', count=3688),
 Row(sampleId='CPCT02030278T', count=4014),
 Row(sampleId='CPCT02110078T', count=4208),
 Row(sampleId='CPCT02030408T', count=1757),
 Row(sampleId='CPCT02300026T', count=5181),
 Row(sampleId='CPCT02020659T', count=2102),
 Row(sampleId='CPCT02120153T', count=3490),
 Row(sampleId='CPCT02040192T', count=2884),
 Row(sampleId='CPCT02220079T', count=3945),
 Row(sampleId='CPCT02060120T', count=7440),
 Row(sampleId='CPCT02330084T', count=2005),
 Row(sampleId='CPCT02030498T', count=1872),
 Row(sampleId='CPCT02080198T', count=1244),
 Row(sampleId='CPCT02020593T', count=4641),
 Row(sampleId='CPCT02070348T', count=3898),
 Row(sampleId='CPCT02100119T', count=3675),
 Row(sampleId='CPCT02080024T', count=3859),
 Row(sampleId='CPCT02020572T', count=4893),
 Row(sampleId='CPCT02290021T', c

In [8]:
%%time
get_to_G_subs(delta_vcf133).select(delta_vcf133.start).collect()[0:10]

CPU times: user 1.81 s, sys: 55.4 ms, total: 1.86 s
Wall time: 5.6 s


[Row(start=134453664),
 Row(start=134541940),
 Row(start=138010940),
 Row(start=140056399),
 Row(start=383409),
 Row(start=430990),
 Row(start=995225),
 Row(start=1383455),
 Row(start=7420713),
 Row(start=8277829)]